# How to create a simulation

This tutorial demonstrates how to create a simulation.

## What is a simulation?

A simulation contains selected sensors, sources to model ray-trace in space.

## Prerequisites

### Perform imports

In [1]:
from pathlib import Path

from ansys.speos.core import Project, Speos, launcher
from ansys.speos.core.simulation import SimulationInteractive, SimulationInverse


### Define constants

The constants help ensure consistency and avoid repetition throughout the example.

In [2]:
HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.
SOURCE_NAME = "Surface.1"
SENSOR_NAME = "Irradiance.1"

## Model Setup

### Load assets
The assets used to run this example are available in the
[PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.

> **Note:** Make sure you
> have downloaded simulation assets and set ``assets_data_path``
> to point to the assets folder.

In [3]:
if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")

### Connect to the RPC Server
This Python client connects to a server where the Speos engine
is running as a service. In this example, the server and
client are the same machine. The launch_local_speos_rpc_method can
be used to start a local instance of the service..

In [4]:
if USE_DOCKER:
    speos = Speos(host=HOSTNAME, port=GRPC_PORT)
else:
    speos = launcher.launch_local_speos_rpc_server(port=GRPC_PORT)

### Create a new project

The only way to create a simulation using the core layer, is to create it from a
project. The ``Project`` class is instantiated by passing a ``Speos`` instance.

In [5]:
p = Project(speos=speos)
print(p)

{
    "name": "",
    "description": "",
    "metadata": {},
    "part_guid": "",
    "sources": [],
    "sensors": [],
    "simulations": [],
    "materials": [],
    "scenes": []
}


### Prepare prerequisites

Create the necessary elements for a simulation: Sensor, source, root part, optical property are
prerequisites.

### Prepare the root part

In [6]:
root_part = p.create_root_part()
body_1 = root_part.create_body(name="Body.1")
face_1 = (
    body_1.create_face(name="Face.1")
    .set_vertices([0, 1, 2, 0, 2, 2, 1, 2, 2])
    .set_facets([0, 1, 2])
    .set_normals([0, 0, 1, 0, 0, 1, 0, 0, 1])
)
root_part.commit()

### Prepare an optical property
Create Optical Property

In [7]:
opt_prop = p.create_optical_property("Material.1")
opt_prop.set_volume_opaque().set_surface_mirror()

Choose the geometry for this optical property : Body.1

In [8]:
opt_prop.set_geometries(geometries=[body_1])
opt_prop.commit()

### Prepare an irradiance sensor

In [9]:
sensor1 = p.create_sensor(name=SENSOR_NAME)
# set type to colorimetric or spectral so that the sensor can be used both in
# direct and inverse simulation
sensor1.set_type_colorimetric()
sensor1.commit()

### Prepare a surface source

In [10]:
source1 = p.create_source(name=SOURCE_NAME)
source1.set_exitance_constant(geometries=[(face_1, True)])
# define a spectrum which is not monochromatic so it can be used in both direct and inverse
# simulation
source1.set_spectrum().set_blackbody()
source1.commit()

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:200: UserWarning: The pySpeos feature : SourceSurface needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SourceSurface(


## Create a simulation

In [11]:
simulation1 = p.create_simulation(name="Simulation.1")
simulation1.set_sensor_paths([SENSOR_NAME]).set_source_paths([SOURCE_NAME])
print(simulation1)
simulation1.commit()
print(simulation1)

local: {
    "name": "Simulation.1",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "metadata": {},
    "simulation_guid": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "ambient_material_uri": "",
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.1",
        "description": "",
        "metadata": {},
        "scene_guid": "b7a2a11e-33bd-421f-94f5-ae90cb485472",
        "simulation_path": "Simulation.1",
        "job_type": "CPU"
    }
}
{
    "name": "Simulation.1",
    "metadata": {
        "Uniqu

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:280: UserWarning: The pySpeos feature : SimulationDirect needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SimulationDirect(
/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/simulation.py:677: UserWarning: Please note that setting a value for light expert option forces a sensorcommit when committing the Simulation class
  self.set_light_expert()


### Set simulation characteristics

Simulation is defined with the same default values as the GUI speos.

If the user would like to modify the simulation characteristics,
it is possible to do so by setting the simulation characteristics as below.

In [12]:
simulation2_direct = p.create_simulation(name="Simulation.2")
simulation2_direct.set_ambient_material_file_uri(
    uri=str(assets_data_path / "AIR.material")
).set_colorimetric_standard_CIE_1964().set_weight_none().set_geom_distance_tolerance(
    0.01
).set_max_impact(200).set_dispersion(False)
simulation2_direct.set_sensor_paths([SENSOR_NAME]).set_source_paths([SOURCE_NAME]).commit()
print(simulation2_direct)

{
    "name": "Simulation.2",
    "metadata": {
        "UniqueId": "1958b1f9-2013-4adf-b2b3-11a8d123b116"
    },
    "simulation_guid": "d7c84951-eb6e-4b59-8e92-941abd6fc546",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 200,
            "colorimetric_standard": "CIE_1964",
            "ambient_material_uri": "/app/assets/AIR.material",
            "dispersion": false,
            "fast_transmission_gathering": false,
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.2",
        "description": "",
        "metadata": {},
        "scene_guid": "b7a2a11e-33bd-421f-94f5-ae90cb485472",
        "simulation_path": "Simulation.2",
        "job_type": "CPU"
    }
}


### Read information

Read simulation information

In [13]:
print(simulation1)

{
    "name": "Simulation.1",
    "metadata": {
        "UniqueId": "e685b763-4367-47d3-8fcb-117195c914f8"
    },
    "simulation_guid": "aea7cb0d-4f0b-4f2b-9a6b-24daf018618a",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "ambient_material_uri": "",
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.1",
        "description": "",
        "metadata": {},
        "scene_guid": "b7a2a11e-33bd-421f-94f5-ae90cb485472",
        "simulation_path": "Simulation.1",
   

Read project information

In [14]:
print(p)

{
    "part_guid": "7473d5b8-09f7-4ff0-b9c0-ee6e737571db",
    "sources": [
        {
            "name": "Surface.1",
            "metadata": {
                "UniqueId": "8e094e9b-34a9-4e1a-9ce9-8058f6b572aa"
            },
            "source_guid": "f91ec535-f5a8-4769-a589-d8468e723b02",
            "description": "",
            "source": {
                "name": "Surface.1",
                "surface": {
                    "luminous_flux": {
                        "luminous_value": 683.0
                    },
                    "intensity_guid": "47c3d3de-4012-4bc4-bd01-ea0eecc12dcd",
                    "exitance_constant": {
                        "geo_paths": [
                            {
                                "geo_path": "Body.1/Face.1",
                                "reverse_normal": true
                            }
                        ]
                    },
                    "spectrum_guid": "163da0fb-286f-4469-ae2b-4504e80e6c25",
             

## Update simulation settings

If you are manipulating a simulation already committed, remember to commit your changes.

If you don't, you will still only watch what is committed on the server.

In [15]:
simulation1.set_ambient_material_file_uri(uri=str(assets_data_path / "AIR.material"))
simulation1.commit()
print(simulation1)

{
    "name": "Simulation.1",
    "metadata": {
        "UniqueId": "e685b763-4367-47d3-8fcb-117195c914f8"
    },
    "simulation_guid": "aea7cb0d-4f0b-4f2b-9a6b-24daf018618a",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "ambient_material_uri": "/app/assets/AIR.material",
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.1",
        "description": "",
        "metadata": {},
        "scene_guid": "b7a2a11e-33bd-421f-94f5-ae90cb485472",
        "simulation_pa

## Reset

Possibility to reset local values from the one available in the server.

In [16]:
simulation1.set_max_impact(1000)  # adjust max impact but no commit
simulation1.reset()  # reset -> this will apply the server value to the local value
simulation1.delete()  # delete (to display the local value with the below print)
print(simulation1)

local: {
    "name": "Simulation.1",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "metadata": {},
    "simulation_guid": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "ambient_material_uri": "/app/assets/AIR.material",
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.1",
        "description": "",
        "metadata": {},
        "scene_guid": "b7a2a11e-33bd-421f-94f5-ae90cb485472",
        "simulation_path": "Simulation.1",
        "job_type": "CPU"
    }
}


## Other simulation examples

### Inverse simulation

In [17]:
simulation3 = p.create_simulation(name="Simulation.3", feature_type=SimulationInverse)
simulation3.set_sensor_paths(sensor_paths=[SENSOR_NAME]).set_source_paths(
    source_paths=[SOURCE_NAME]
).commit()
print(simulation3)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.10/site-packages/ansys/speos/core/project.py:287: UserWarning: The pySpeos feature : SimulationInverse needs a Speos Version of 2025 R2 SP0 or higher.
  feature = SimulationInverse(


{
    "name": "Simulation.3",
    "metadata": {
        "UniqueId": "6f8348f1-5923-4704-b0c1-b6e254e2cca3"
    },
    "simulation_guid": "f2d5e097-a223-448f-9e44-90cdbec258ec",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "simulation": {
        "inverse_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "number_of_gathering_rays_per_source": 1,
            "colorimetric_standard": "CIE_1931",
            "dispersion": false,
            "splitting": false,
            "maximum_gathering_error": 0,
            "maximum_gathering_error_percentage": 0.0,
            "fast_transmission_gathering": false,
            "ambient_material_uri": "",
            "optimized_propagation_none": {
                "stop_condition_passes_number": 5
            },
     

### Interactive simulation

In [18]:
simulation4 = p.create_simulation(name="Simulation.4", feature_type=SimulationInteractive)
simulation4.set_source_paths(source_paths=[SOURCE_NAME]).commit()
print(simulation4)

{
    "name": "Simulation.4",
    "metadata": {
        "UniqueId": "38fd90a3-2ce8-4d50-a267-37125f233a14"
    },
    "simulation_guid": "ebad3c2c-4161-4305-a6b9-07205f8d0d81",
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "sensor_paths": [],
    "simulation": {
        "name": "Simulation.4",
        "interactive_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "colorimetric_standard": "CIE_1931",
            "ambient_material_uri": "",
            "rays_number_per_sources": [],
            "light_expert": false,
            "impact_report": false
        },
        "description": "",
        "metadata": {},
        "scene_guid": "b7a2a11e-33bd-421f-94f5-ae90cb485472",
        "simulation_path": "Simulation.4",
        "job_type": "CPU"
    }
}


In [19]:
speos.close()

True